# Scenario 5: Comparison of modelling tools

This notebook is part of the publication "EnzymeML at Work" from Lauterbach et al. 2022 and compares the fitting of a micro-kinetic model (specified in an EnzymeML document) to experimental data (specified in the same EnzymeML document). 

Generation of the EnzymeML document and individual fitting of the data with either PySCeS or COPASI have been dealt with in separate notebooks.

## Prerequisites

For the kinetic modelling to work, you need to have `PyEnzyme`, `Basico` and `PySCeS` installed. This can be achieved with:

```
    !pip install copasi-basico
    !pip install pysces
    !pip install git+git://github.com/EnzymeML/PyEnzyme.git@Refactoring
```

This is **not needed** when running this notebook via **Binder**, as the environment is already set up.

For the parameter estimation with PySCeS, additionally the CVODE algorithm is needed; this is provided by **Assimulo**. If you are using the **Anaconda** Python Distribution (and when running this notebook via **Binder**), this can easily be achieved by uncommenting and running the following line of code. Alternatively, refer to the Assimulo documentation: https://jmodelica.org/assimulo/

In [1]:
# !conda install -y -c conda-forge assimulo

-----
## Comparison of modelling with PySCeS and COPASI

In separate notebooks, the EnzymeMLDocument that had been adapted to the micro-kinetic model was previously modeled and optimized using PySCeS and COPASI. Since both modeling package interfaces are an integral part of PyEnzyme (linked via an interface called Thin Layer), a simple call to the corresponding Thin Layer object is necessary.

In [2]:
# Load the EnzymeML Document from file
from pyenzyme import EnzymeMLDocument

enzmldoc = EnzymeMLDocument.fromFile("Model_4.omex")

### Modelling with the PySCeS thin layer

Thin Layers require to follow a given metaclass and thus the syntax of every modeling layer follows the Initialization > ```optimize```-method > ```write```-method procedure. 

In [3]:
from pyenzyme.thinlayers import ThinLayerPysces

Before optimization, it might be necessary to define initial values. Since manipulating the KineticParameter initial_values attributes inside the script that generates the EnzymeMLDocument can get quite tedious, PyEnzyme offers an external data structure from within initial values can be applied. This way, the EnzymeML document is only modifed at optimization and remains untouched until then.

The initialization file is in the YAML format and contains all reactions and their parameters together with the initial estimates for the parameter valuess. 

In [4]:
# Initialize the layer
tl_pysces = ThinLayerPysces(
    "Model_4.omex", 
    init_file="EnzymeML_Lagerman_init_values_.yaml",
    model_dir="pySCeS"
)


Check SBML support is at action level 2
SBML file is L3V2



*********ERRORS***********


*********ERRORS***********


Possible errors detected in SBML conversion, Model may be incomplete. Please check the error log file "EnzymeML_Lagerman.xml-sbml_conversion_errors.txt" for details.


*******************************************************************
Issues encountered in SBML translation (model processed anyway)
SBML source: pySCeS/EnzymeML_Lagerman.xml
*******************************************************************

Parameter units ignored for parameters:
['v_r', 'K_si', 'K_n'] 

Parameter units ignored for (local) parameters:
['K_s', 'k_2', 'k_6', 'k_3', 'K_pg', 'k_5', 'k_4', 'k_4b', 'K_p', 'k_d'] 

*******************************************************************

Info: single compartment model: locating "r0" in default compartment
Info: single compartment model: locating "r1" in default compartment
Info: single compartment model: locating "r2" in default compartment
Info:

In [5]:
# Run optimization
tl_pysces.model.mode_integrator='CVODE'
tl_opt = tl_pysces.optimize(method="least_squares")

# Write to new EnzymeMLDocument and save
pysces_doc = tl_pysces.write()
pysces_doc.toFile(".", name="EnzymeML_Lagerman_M4_PySCeS_Modeled")


Archive was written to ./EnzymeML_Lagerman_M4_PySCeS_Modeled.omex



### Modelling with the COPASI thin layer

In the same manner the COPASI Thin Layer can be used to model the given data. 

The COPASI optimization is set up to use the same initial values and the same fitting algorithm that was used with PySCeS, to allow an easy comparison.

In [6]:
from pyenzyme.thinlayers import ThinLayerCopasi

In [7]:
# Initialize COPASI Thin Layer
tl_copasi = ThinLayerCopasi(
    "Model_4.omex", "COPASI",
    init_file="EnzymeML_Lagerman_init_values_.yaml"
)

tl_copasi.optimize()

,start,lower,upper,reaction_id,value,std_deviation
name,,,,,,
K_si,6.346729e+00,0.010,1000.0,None,6.309752e+00,0.283728
K_n,1.100543e+01,0.010,10000.0,None,1.099344e+01,7.750598
K_s,5.676256e+00,0.010,1000.0,r1,5.669943e+00,3.738394
k_2,6.520855e+05,1.000,1000000.0,r2,6.516895e+05,23431.513796
k_6,3.931694e+05,1.000,10000000.0,r5,3.923263e+05,262196.751603
k_3,8.938179e+00,1.000,1000000.0,r6,8.745659e+00,287.342389
K_pg,4.734446e+01,0.010,1000.0,r7,4.870176e+01,464.369474
k_5,6.722958e+05,1.000,1000000.0,r9,6.691782e+05,70546.311658
k_4,1.870571e+06,1.000,100000000.0,r10,1.866817e+06,68474.213674


In [8]:
copasi_doc = tl_copasi.write()
copasi_doc.toFile(".", name="EnzymeML_Lagerman_M4_COPASI_Modeled")


Archive was written to ./EnzymeML_Lagerman_M4_COPASI_Modeled.omex



### Comparison of results

Both results can now be compared by individually exporting the estimated parameters using the ```exportKineticParameters```-method found in the ```EnzymeMLDocument``` instance that returns a Pandas ```DataFrame``` object. Finally, for the sake of comparison, both result are merged into a single ```DataFrame```.

In [9]:
params = pysces_doc.exportKineticParameters(exclude_constant=True)
params.rename({"value": "PySCeS"}, axis="columns", inplace=True)
params["COPASI"] = copasi_doc.exportKineticParameters(exclude_constant=True).value

params[["name", "PySCeS", "COPASI", "unit"]]

,name,PySCeS,COPASI,unit
reaction,,,,
r1,K_s,5.129503e+00,5.669943e+00,mmole / l
r2,k_2,5.694527e+05,6.516895e+05,1 / min
r5,k_6,2.495553e+05,3.923263e+05,1 / min
r6,k_3,1.589958e+01,8.745659e+00,1 / min
r7,K_pg,1.299523e+02,4.870176e+01,mmole / l
r9,k_5,8.846055e+05,6.691782e+05,1 / min
r10,k_4,1.577461e+06,1.866817e+06,1 / min
r10,k_4b,3.680255e+04,4.253530e+04,1 / min
r11,K_p,1.295747e+00,9.339814e-01,mmole / l


-------